![TraderPy Logo](https://traderpy.files.wordpress.com/2021/01/cropped-logo3.png)

# Code an RSI Trading Startegy in Python

**Author**: TraderPy - Algorithmic Trading (Tu Khac Nguyen)

**Youtube**: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ


## DISCLAIMER

Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

---

## Introduction

In this Notebook, we will code and backtest an RSI Trading Strategy. RSI is a Technical Indicator used to find oversold and overbought prices and therefore is popular for mean reversion strategies

## Strategy Description

We will trade on the EURUSD daily timeframe. When the RSI indicator drops below 30 (oversold), we will buy and when RSI exceeds above 70 (overbought), we will sell.

For Exit, we will use the ATR Indicator. ATR measures the average range for each period and is therefore a good volatility indicator. After entering a trade, we will set take profit and stop loss to 2 ATR each

---

In [1]:
# Libraries 
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px

In [2]:
pip install MetaTrader5

In [10]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [14]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_H1
start_pos = 0
num_bars = 5000

## Retrieving Historical Prices

In [15]:
# Requesting historical data
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close']]
df['time'] = pd.to_datetime(df['time'], unit='s')

df

,time,open,high,low,close
0,2021-11-08 02:00:00,1.15680,1.15680,1.15582,1.15604
1,2021-11-08 03:00:00,1.15604,1.15624,1.15539,1.15548
2,2021-11-08 04:00:00,1.15548,1.15593,1.15532,1.15593
3,2021-11-08 05:00:00,1.15593,1.15623,1.15574,1.15577
4,2021-11-08 06:00:00,1.15578,1.15634,1.15566,1.15610
...,...,...,...,...,...
4995,2022-08-25 10:00:00,1.00106,1.00237,0.99953,0.99996
4996,2022-08-25 11:00:00,0.99994,1.00122,0.99871,1.00072
4997,2022-08-25 12:00:00,1.00074,1.00113,0.99868,0.99972
4998,2022-08-25 13:00:00,0.99972,0.99979,0.99741,0.99803


In [17]:
df.to_csv("eurusd_h1_5000.csv")

## Plotting EURUSD Close Prices

In [16]:
fig = px.line(df, x='time', y='close', title='EURUSD - Close Prices')  # creating a figure using px.line
display(fig)  # showing figure in output

## Importing classes for Backtesting

In [ ]:
# class Position contain data about trades opened/closed during the backtest 
class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        

# class Strategy defines trading logic and evaluates the backtest based on opened/closed positions
class Strategy:
    def __init__(self, df, starting_balance):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df
    
    # return backtest result
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
    
    # add Position class to list
    def add_position(self, position):
        self.positions.append(position)
        return True
    
    # close positions when stop loss or take profit is reached
    def close_tp_sl(self, data):
        for pos in self.positions:
                    if pos.status == 'open':
                        if (pos.sl >= data.close and pos.order_type == 'buy'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.sl <= data.close and pos.order_type == 'sell'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.tp <= data.close and pos.order_type == 'buy'):
                            pos.close_position(data.time, pos.tp)
                        elif (pos.tp >= data.close and pos.order_type == 'sell'):
                            pos.close_position(data.time, pos.tp)
                            
    # check for open positions
    def has_open_positions(self):
        for pos in self.positions:
            if pos.status == 'open':
                return True
        return False
    
    # strategy logic how positions should be opened/closed
    def logic(self, data):
        
        # if no position is open
        if not self.has_open_positions():
            
            # if RSI less then 30 -> BUY
            if data['rsi_14'] < 30:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['close']
                order_type = 'buy'
                volume = 10000
                sl = open_price - 2 * data['atr_14']
                tp = open_price + 2 * data['atr_14']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
            
            # if RSI greater than 70 -> SELL
            elif data['rsi_14'] > 70:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['close']
                order_type = 'sell'
                volume = 10000
                sl = open_price + 2 * data['atr_14']
                tp = open_price - 2 * data['atr_14']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
        
        
# logic
    def run(self):
        # data represents a moment in time while iterating through the backtest
        for i, data in self.data.iterrows():
            # close positions when stop loss or take profit is reached
            self.close_tp_sl(data)
            
            # strategy logic
            self.logic(data)
        
        return self.get_positions_df()

In [ ]:
# preparing data for backtest
backtest_df = df[14:]  # removing NaN values
backtest_df

,time,open,high,low,close,gain,loss,ema_gain,ema_loss,rs,rsi_14,range,atr_14
14,2018-02-05,1.24310,1.24750,1.23626,1.23664,0.00000,0.00646,0.002419,0.002255,1.072967,51.759962,0.01124,0.011084
15,2018-02-06,1.23657,1.24348,1.23140,1.23764,0.00107,0.00000,0.002219,0.001920,1.155633,53.609919,0.01208,0.010904
16,2018-02-07,1.23755,1.24062,1.22459,1.22646,0.00000,0.01109,0.001895,0.003261,0.581126,36.753947,0.01603,0.011334
17,2018-02-08,1.22621,1.22951,1.22122,1.22465,0.00000,0.00156,0.001621,0.003015,0.537737,34.969361,0.00829,0.011350
18,2018-02-09,1.22456,1.22873,1.22056,1.22518,0.00062,0.00000,0.001478,0.002585,0.571976,36.385815,0.00817,0.011517
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-11-17,1.13203,1.13323,1.12635,1.13189,0.00000,0.00014,0.000740,0.003024,0.244790,19.665181,0.00688,0.007051
996,2021-11-18,1.13191,1.13742,1.13138,1.13729,0.00538,0.00000,0.001359,0.002620,0.518541,34.147333,0.00604,0.006376
997,2021-11-19,1.13673,1.13727,1.12501,1.12820,0.00000,0.00853,0.001178,0.003408,0.345508,25.678646,0.01226,0.006801
998,2021-11-22,1.12911,1.12911,1.12308,1.12338,0.00000,0.00573,0.001021,0.003718,0.274509,21.538400,0.00603,0.006959


## Running the backtest

In [ ]:
# creating an instance of Strategy class
rsi_strategy = Strategy(backtest_df, 10000)

# running the backtest
backtest_result = rsi_strategy.run()

backtest_result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2018-02-15,1.25071,sell,10000,1.272256,1.229164,2018-02-21,1.229164,215.457143,closed,10215.457143
1,2018-04-25,1.21610,buy,10000,1.201763,1.230437,2018-05-01,1.201763,-143.371429,closed,10072.085714
2,2018-05-01,1.19920,buy,10000,1.183463,1.214937,2018-05-16,1.183463,-157.371429,closed,9914.714286
3,2018-05-16,1.18079,buy,10000,1.163506,1.198074,2018-05-28,1.163506,-172.842857,closed,9741.871429
4,2018-05-28,1.16242,buy,10000,1.145244,1.179596,2018-06-07,1.179596,171.757143,closed,9913.628571
5,2018-08-10,1.14093,buy,10000,1.126721,1.155139,2018-08-21,1.155139,142.085714,closed,10055.714286
6,2018-08-27,1.16784,sell,10000,1.185381,1.150299,2018-10-03,1.150299,175.414286,closed,10231.128571
7,2018-10-03,1.14784,buy,10000,1.130780,1.164900,2018-11-12,1.130780,-170.600000,closed,10060.528571
8,2018-11-12,1.12174,buy,10000,1.105926,1.137554,2018-11-16,1.137554,158.142857,closed,10218.671429
9,2019-02-11,1.12748,buy,10000,1.115411,1.139549,2019-03-20,1.139549,120.685714,closed,10339.357143


## Visualizing the Backtest

In [ ]:
# analysing closed positions only
backtest_result = backtest_result[backtest_result['status'] == 'closed']

# visualizing trades
fig_backtest = px.line(df, x='time', y=['close'], title='RSI Strategy - Trades')

# adding trades to plots
for i, position in backtest_result.iterrows():
    if position.status == 'closed':
        fig_backtest.add_shape(type="line",
            x0=position.open_datetime, y0=position.open_price, x1=position.close_datetime, y1=position.close_price,
            line=dict(
                color="green" if position.profit >= 0 else "red",
                width=3)
            )

fig_backtest

## Plotting PnL

In [ ]:
fig_pnl = px.line(backtest_result, x='close_datetime', y='pnl')
fig_pnl